This notebook shows the process of obtaining and saving the embeddings for category terms in the candidate category list. 

**This only needs to be done once. The embeddings are saved as a looking-up table for searching for similar categories later.**

In [1]:
pip install transformers

     |████████████████████████████████| 2.8 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 47.6 MB/s 
     |████████████████████████████████| 3.3 MB 28.9 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 636 kB 47.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
from numpy import savetxt
import torch
import math

## Define model

We try two different model, one is BERT and the other one is ClinicalBERT

In [3]:
BERT = 'sentence-transformers/bert-base-nli-mean-tokens'
BERT_tokenizer = AutoTokenizer.from_pretrained(BERT)
BERT_model = AutoModel.from_pretrained(BERT)

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [4]:
ClinicalBERT = 'emilyalsentzer/Bio_ClinicalBERT'
ClinicalBERT_tokenizer = AutoTokenizer.from_pretrained(ClinicalBERT)
ClinicalBERT_model = AutoModel.from_pretrained(ClinicalBERT)

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Load categories list 

load pre-defined category list

In [5]:
url = 'https://raw.githubusercontent.com/casszhao/FAIR/main/sources/0901_full_list.csv'
sorted_cat = pd.read_csv(url, header=None)

In [6]:
sorted_cat = sorted_cat[0].to_list()
sorted_cat = list(dict.fromkeys(sorted_cat))

## Get embeddings

In [7]:
# process batch, get embeddings for a batch
def get_mean_pooled_np_batch(sorted_cat, model_name, tokenizer_name):

  model = model_name
  tokenizer = tokenizer_name

  tokens = {'input_ids': [], 'attention_mask': []}

  for sentence in sorted_cat:
      # encode each sentence and append to dictionary
      new_tokens = tokenizer.encode_plus(sentence, max_length=20,
                                        truncation=True, padding='max_length',
                                        return_tensors='pt')
      tokens['input_ids'].append(new_tokens['input_ids'][0])
      tokens['attention_mask'].append(new_tokens['attention_mask'][0])

  # reformat list of tensors into single tensor, contain all sents
  tokens['input_ids'] = torch.stack(tokens['input_ids'])
  tokens['attention_mask'] = torch.stack(tokens['attention_mask'])
  outputs = model(**tokens)
  embeddings = outputs.last_hidden_state

  attention_mask = tokens['attention_mask']
  mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
  masked_embeddings = embeddings * mask
  summed = torch.sum(masked_embeddings, 1)

  summed_mask = torch.clamp(mask.sum(1), min=1e-9)
  mean_pooled = summed / summed_mask
  return mean_pooled.detach().numpy()

for the memory reason, we process data by batch, each batch is 10

In [8]:
total_batch = math.ceil(len(sorted_cat)/10)

In [9]:
# batch in batch out to save ram
for i in range(total_batch):
  temp_list = sorted_cat[i*10:(i+1)*10]

  if i == 0:
    BERT_embeddings = get_mean_pooled_np_batch(temp_list, BERT_model, BERT_tokenizer)
    ClinicalBERT_embeddings = get_mean_pooled_np_batch(temp_list, ClinicalBERT_model, ClinicalBERT_tokenizer)
  else:
    temp_BERT_embeddings = get_mean_pooled_np_batch(temp_list, BERT_model, BERT_tokenizer)
    temp_ClinicalBERT_embeddings = get_mean_pooled_np_batch(temp_list, ClinicalBERT_model, ClinicalBERT_tokenizer)

    BERT_embeddings = np.concatenate((BERT_embeddings, temp_BERT_embeddings), axis=0)
    ClinicalBERT_embeddings = np.concatenate((ClinicalBERT_embeddings, temp_ClinicalBERT_embeddings), axis=0)

In [10]:
import pickle
with open('BERT_embeddings.pkl','wb') as f:
  pickle.dump(BERT_embeddings, f)

with open('ClinicalBERT_embeddings.pkl','wb') as f:
  pickle.dump(ClinicalBERT_embeddings, f)